In [1]:
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [14]:
!ffmpeg -i 1.mp3 -acodec pcm_s16le -ac 1 -ar 16000 out1.wav

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [15]:
!ffmpeg -i 2.mp3 -acodec pcm_s16le -ac 1 -ar 16000 out2.wav

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [5]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 122kB/s 


In [10]:
!pip install pydub

In [0]:
# Import necessary libraries 
from pydub import AudioSegment 
import speech_recognition as sr 

In [16]:
# Input audio file to be sliced 
audio = AudioSegment.from_wav("/content/out1.wav") 

''' 
Step #1 - Slicing the audio file into smaller chunks. 
'''
# Length of the audiofile in milliseconds 
n = len(audio) 

# Variable to count the number of sliced chunks 
counter = 1

# Text file to write the recognized audio 
fh = open("recognized1.txt", "w+") 

# Interval length at which to slice the audio file. 
# If length is 22 seconds, and interval is 5 seconds, 
# The chunks created will be: 
# chunk1 : 0 - 5 seconds 
# chunk2 : 5 - 10 seconds 
# chunk3 : 10 - 15 seconds 
# chunk4 : 15 - 20 seconds 
# chunk5 : 20 - 22 seconds 
interval = 5 * 1000

# Length of audio to overlap. 
# If length is 22 seconds, and interval is 5 seconds, 
# With overlap as 1.5 seconds, 
# The chunks created will be: 
# chunk1 : 0 - 5 seconds 
# chunk2 : 3.5 - 8.5 seconds 
# chunk3 : 7 - 12 seconds 
# chunk4 : 10.5 - 15.5 seconds 
# chunk5 : 14 - 19.5 seconds 
# chunk6 : 18 - 22 seconds 
overlap = 1.5 * 0  #creating overlap equals to zero

# Initialize start and end seconds to 0 
start = 0
end = 0

# Flag to keep track of end of file. 
# When audio reaches its end, flag is set to 1 and we break 
flag = 0

# Iterate from 0 to end of the file, 
# with increment = interval 
for i in range(0, 2 * n, interval): 
	
	# During first iteration, 
	# start is 0, end is the interval 
	if i == 0: 
		start = 0
		end = interval 

	# All other iterations, 
	# start is the previous end - overlap 
	# end becomes end + interval 
	else: 
		start = end - overlap 
		end = start + interval 

	# When end becomes greater than the file length, 
	# end is set to the file length 
	# flag is set to 1 to indicate break. 
	if end >= n: 
		end = n 
		flag = 1

	# Storing audio file from the defined start to end 
	chunk = audio[start:end] 

	# Filename / Path to store the sliced audio 
	filename = 'a'+str(counter)+'.wav'

	# Store the sliced audio file to the defined path 
	chunk.export(filename, format ="wav") 
	# Print information about the current chunk 
	print("Processing chunk "+str(counter)+". Start = "
						+str(start)+" end = "+str(end)) 

	# Increment counter for the next chunk 
	counter = counter + 1
	
	# Slicing of the audio file is done. 
	# Skip the below steps if there is some other usage 
	# for the sliced audio files. 

  #''' 
  #Step #2 - Recognizing the chunk and writing to a file. 
  #'''

	# Here, Google Speech Recognition is used 
	# to take each chunk and recognize the text in it. 

	# Specify the audio file to recognize 

	AUDIO_FILE = filename 

	# Initialize the recognizer 
	r = sr.Recognizer() 

	# Traverse the audio file and listen to the audio 
	with sr.AudioFile(AUDIO_FILE) as source: 
		audio_listened = r.listen(source) 

	# Try to recognize the listened audio 
	# And catch expections. 
	try:	 
		rec = r.recognize_google(audio_listened) 
		
		# If recognized, write into the file. 
		fh.write(rec+" ") 
	
	# If google could not understand the audio 
	except sr.UnknownValueError: 
		print("Could not understand audio") 

	# If the results cannot be requested from Google. 
	# Probably an internet connection error. 
	except sr.RequestError as e: 
		print("Could not request results.") 

	# Check for flag. 
	# If flag is 1, end of the whole audio reached. 
	# Close the file and break. 
	if flag == 1: 
		fh.close() 
		break

Processing chunk 1. Start = 0 end = 5000
Processing chunk 2. Start = 5000.0 end = 9648
Could not understand audio


In [17]:
# Input audio file to be sliced 
audio = AudioSegment.from_wav("/content/out2.wav") 

''' 
Step #1 - Slicing the audio file into smaller chunks. 
'''
# Length of the audiofile in milliseconds 
n = len(audio) 

# Variable to count the number of sliced chunks 
counter = 1

# Text file to write the recognized audio 
fh = open("recognized2.txt", "w+") 

# Interval length at which to slice the audio file. 
# If length is 22 seconds, and interval is 5 seconds, 
# The chunks created will be: 
# chunk1 : 0 - 5 seconds 
# chunk2 : 5 - 10 seconds 
# chunk3 : 10 - 15 seconds 
# chunk4 : 15 - 20 seconds 
# chunk5 : 20 - 22 seconds 
interval = 5 * 1000

# Length of audio to overlap. 
# If length is 22 seconds, and interval is 5 seconds, 
# With overlap as 1.5 seconds, 
# The chunks created will be: 
# chunk1 : 0 - 5 seconds 
# chunk2 : 3.5 - 8.5 seconds 
# chunk3 : 7 - 12 seconds 
# chunk4 : 10.5 - 15.5 seconds 
# chunk5 : 14 - 19.5 seconds 
# chunk6 : 18 - 22 seconds 
overlap = 1.5 * 0  #creating overlap equals to zero

# Initialize start and end seconds to 0 
start = 0
end = 0

# Flag to keep track of end of file. 
# When audio reaches its end, flag is set to 1 and we break 
flag = 0

# Iterate from 0 to end of the file, 
# with increment = interval 
for i in range(0, 2 * n, interval): 
	
	# During first iteration, 
	# start is 0, end is the interval 
	if i == 0: 
		start = 0
		end = interval 

	# All other iterations, 
	# start is the previous end - overlap 
	# end becomes end + interval 
	else: 
		start = end - overlap 
		end = start + interval 

	# When end becomes greater than the file length, 
	# end is set to the file length 
	# flag is set to 1 to indicate break. 
	if end >= n: 
		end = n 
		flag = 1

	# Storing audio file from the defined start to end 
	chunk = audio[start:end] 

	# Filename / Path to store the sliced audio 
	filename = 'b'+str(counter)+'.wav'

	# Store the sliced audio file to the defined path 
	chunk.export(filename, format ="wav") 
	# Print information about the current chunk 
	print("Processing chunk "+str(counter)+". Start = "
						+str(start)+" end = "+str(end)) 

	# Increment counter for the next chunk 
	counter = counter + 1
	
	# Slicing of the audio file is done. 
	# Skip the below steps if there is some other usage 
	# for the sliced audio files. 

  #''' 
  #Step #2 - Recognizing the chunk and writing to a file. 
  #'''

	# Here, Google Speech Recognition is used 
	# to take each chunk and recognize the text in it. 

	# Specify the audio file to recognize 

	AUDIO_FILE = filename 

	# Initialize the recognizer 
	r = sr.Recognizer() 

	# Traverse the audio file and listen to the audio 
	with sr.AudioFile(AUDIO_FILE) as source: 
		audio_listened = r.listen(source) 

	# Try to recognize the listened audio 
	# And catch expections. 
	try:	 
		rec = r.recognize_google(audio_listened) 
		
		# If recognized, write into the file. 
		fh.write(rec+" ") 
	
	# If google could not understand the audio 
	except sr.UnknownValueError: 
		print("Could not understand audio") 

	# If the results cannot be requested from Google. 
	# Probably an internet connection error. 
	except sr.RequestError as e: 
		print("Could not request results.") 

	# Check for flag. 
	# If flag is 1, end of the whole audio reached. 
	# Close the file and break. 
	if flag == 1: 
		fh.close() 
		break

Processing chunk 1. Start = 0 end = 5000
Processing chunk 2. Start = 5000.0 end = 10000.0
Processing chunk 3. Start = 10000.0 end = 15000.0
Processing chunk 4. Start = 15000.0 end = 20000.0
Processing chunk 5. Start = 20000.0 end = 23832


In [0]:
sentence_1 = ""
with open('recognized1.txt', 'r') as fh:
    for line in fh:
        sentence_1 = sentence_1 + " " + line

In [0]:
sentence_2 = ""
with open('recognized2.txt', 'r') as fh:
    for line in fh:
        sentence_2 = sentence_2 + " " + line

In [22]:
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [0]:
messages = [sentence_1, sentence_2]

In [27]:
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [28]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})

    corr = np.inner(message_embeddings_, message_embeddings_)
    print(corr[0][1])

0.45126766
